# Modelo MLP para clasificación en CIFAR-100

## Mapeo de clases a superclases

In [1]:
class_mapping = {
    'aquatic mammals': ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
    'fish': ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
    'flowers': ['orchids', 'poppies', 'roses', 'sunflowers', 'tulips'],
    'food containers': ['bottles', 'bowls', 'cans', 'cups', 'plates'],
    'fruit and vegetables': ['apples', 'mushrooms', 'oranges', 'pears', 'sweet_peppers'],
    'household electrical devices': ['clock', 'computer_keyboard', 'lamp', 'telephone', 'television'],
    'household furniture': ['bed', 'chair', 'couch', 'table', 'wardrobe'],
    'insects': ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
    'large carnivores': ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
    'large man-made outdoor things': ['bridge', 'castle', 'house', 'road', 'skyscraper'],
    'large natural outdoor scenes': ['cloud', 'forest', 'mountain', 'plain', 'sea'],
    'large omnivores and herbivores': ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
    'medium-sized mammals': ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
    'non-insect invertebrates': ['crab', 'lobster', 'snail', 'spider', 'worm'],
    'people': ['baby', 'boy', 'girl', 'man', 'woman'],
    'reptiles': ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
    'small mammals': ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
    'trees': ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
    'vehicles 1': ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
    'vehicles 2': ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor']
}

## Importaciones

In [15]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeNormal
from sklearn.model_selection import train_test_split
import numpy as np

## Cargar datos CIFAR-100 (fine y coarse)

In [16]:
(x_train, y_train_fine), (x_test, y_test_fine) = cifar100.load_data(label_mode='fine')
(_, y_train_coarse), (_, y_test_coarse) = cifar100.load_data(label_mode='coarse')

## Normalizar imágenes

In [17]:
def normalize_images(images):
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2470, 0.2435, 0.2616])
    return (images.astype('float32') - mean) / std

x_train = normalize_images(x_train)
x_test = normalize_images(x_test)

## One-hot encoding

In [19]:
y_train_fine = to_categorical(y_train_fine, 100)
y_test_fine = to_categorical(y_test_fine, 100)
y_train_coarse = to_categorical(y_train_coarse, 20)
y_test_coarse = to_categorical(y_test_coarse, 20)

### Separar validación

In [20]:
x_train_split, x_val_split, y_train_fine_split, y_val_fine_split, y_train_coarse_split, y_val_coarse_split = train_test_split(
    x_train, y_train_fine, y_train_coarse, test_size=0.15, random_state=42)

### Definir la clase MultiOutputDataGenerator

In [21]:
class MultiOutputDataGenerator(Sequence):
    def __init__(self, x, y_fine, y_coarse, batch_size, datagen, shuffle=True):
        self.x = x
        self.y_fine = y_fine
        self.y_coarse = y_coarse
        self.batch_size = batch_size
        self.datagen = datagen
        self.shuffle = shuffle
        self.indexes = np.arange(len(x))
        if shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        x_batch = self.x[batch_indexes]
        y_fine_batch = self.y_fine[batch_indexes]
        y_coarse_batch = self.y_coarse[batch_indexes]
        
        # Aplicar data augmentation
        x_batch_augmented = np.zeros_like(x_batch)
        for i in range(len(x_batch)):
            x_batch_augmented[i] = self.datagen.random_transform(x_batch[i])
        
        return x_batch_augmented, {'fine_output': y_fine_batch, 'coarse_output': y_coarse_batch}
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Crear el generador de datos
train_generator = MultiOutputDataGenerator(
    x_train_split,
    y_train_fine_split,
    y_train_coarse_split,
    batch_size=128,
    datagen=datagen,
    shuffle=True
)

### Data augmentation

In [22]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='constant',
    cval=0
)

## Arquitectura Modelo MLP

In [9]:
input_shape = (32, 32, 3)
inputs = Input(shape=input_shape)
x = Flatten()(inputs)

# Capas ocultas con inicialización He y regularización L1-L2
x = Dense(2048, activation='relu', 
          kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
          kernel_initializer=HeNormal())(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(1536, activation='relu',
          kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
          kernel_initializer=HeNormal())(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(1024, activation='relu',
          kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
          kernel_initializer=HeNormal())(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(768, activation='relu',
          kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
          kernel_initializer=HeNormal())(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Capas de salida con label smoothing implícito
output_fine = Dense(100, activation='softmax', name='fine_output')(x)
output_coarse = Dense(20, activation='softmax', name='coarse_output')(x)

model = Model(inputs=inputs, outputs=[output_fine, output_coarse])

### Optimizador mejorado

In [26]:
def lr_schedule(epoch):
    """Learning Rate Schedule"""
    lr = 3e-4
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 60:
        lr *= 1e-3
    elif epoch > 40:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    return lr

### Callbacks

In [27]:
early_stopping = EarlyStopping(
    monitor='val_fine_output_accuracy',
    patience=10,
    restore_best_weights=True,
    min_delta=0.001,
    mode='max'
)

checkpoint = ModelCheckpoint(
    'best_mlp_cifar100.h5',
    monitor='val_fine_output_accuracy',
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_fine_output_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

lr_scheduler = LearningRateScheduler(lr_schedule)

### Optimizador Nadam (Adam con Nesterov momentum)

In [28]:
optimizer = Nadam(
    learning_rate=3e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-8
)

model.compile(
    optimizer=optimizer,
    loss={'fine_output': 'categorical_crossentropy', 'coarse_output': 'categorical_crossentropy'},
    metrics={'fine_output': 'accuracy', 'coarse_output': 'accuracy'},
    loss_weights={'fine_output': 0.7, 'coarse_output': 0.3}
)

### Entrenamiento del modelo

In [29]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=150,
    validation_data=(x_val_split, {'fine_output': y_val_fine_split, 'coarse_output': y_val_coarse_split}),
    callbacks=[early_stopping, checkpoint, reduce_lr, lr_scheduler],
    verbose=1
)

Epoch 1/150
333/333 [==============================] - 94s 273ms/step - loss: 4.1048 - fine_output_loss: 3.4771 - coarse_output_loss: 2.3045 - fine_output_accuracy: 0.1672 - coarse_output_accuracy: 0.2840 - val_loss: 4.0854 - val_fine_output_loss: 3.4755 - val_coarse_output_loss: 2.2781 - val_fine_output_accuracy: 0.1661 - val_coarse_output_accuracy: 0.2827 - lr: 3.0000e-04
Epoch 2/150
333/333 [==============================] - 95s 286ms/step - loss: 4.0966 - fine_output_loss: 3.4907 - coarse_output_loss: 2.3095 - fine_output_accuracy: 0.1649 - coarse_output_accuracy: 0.2873 - val_loss: 4.2408 - val_fine_output_loss: 3.6677 - val_coarse_output_loss: 2.3972 - val_fine_output_accuracy: 0.1368 - val_coarse_output_accuracy: 0.2521 - lr: 3.0000e-04
Epoch 3/150
333/333 [==============================] - 98s 294ms/step - loss: 4.0685 - fine_output_loss: 3.4788 - coarse_output_loss: 2.3065 - fine_output_accuracy: 0.1658 - coarse_output_accuracy: 0.2833 - val_loss: 3.9974 - val_fine_output_loss

## Evaluación